In [1]:
# Vérification de la version d'Evidently

import sys

# Méthode 1 : Version d'Evidently
try:
    import evidently
    print(f"✅ Evidently version: {evidently.__version__}")
except ImportError:
    print("❌ Evidently n'est pas installé")
except AttributeError:
    print("⚠️ Impossible de récupérer la version")
    
# Méthode 2 : Via pip (plus fiable)
try:
    import pkg_resources
    version = pkg_resources.get_distribution("evidently").version
    print(f"✅ Evidently version (via pip): {version}")
except:
    print("⚠️ Impossible de récupérer la version via pkg_resources")

# Méthode 3 : Via importlib.metadata (Python 3.8+)
try:
    from importlib.metadata import version as get_version
    version = get_version("evidently")
    print(f"✅ Evidently version (via importlib): {version}")
except:
    print("⚠️ Impossible de récupérer la version via importlib")

# Vérification des dépendances clés
print("\n" + "="*50)
print("Vérification des modules Evidently utilisés :")
print("="*50)

modules_to_check = [
    "evidently.report",
    "evidently.metric_preset",
    "evidently.test_suite",
    "evidently.test_preset",
    "evidently.tests"
]

for module in modules_to_check:
    try:
        __import__(module)
        print(f"✅ {module}")
    except ImportError as e:
        print(f"❌ {module} - {e}")

# Info Python
print("\n" + "="*50)
print(f"Python version: {sys.version}")
print("="*50)

✅ Evidently version: 0.4.33
✅ Evidently version (via pip): 0.4.33
✅ Evidently version (via importlib): 0.4.33

Vérification des modules Evidently utilisés :


/var/folders/jj/3s72m7nx3039djzxqt_p86w80000gn/T/ipykernel_1535/1749556173.py:16: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


✅ evidently.report
✅ evidently.metric_preset
✅ evidently.test_suite
✅ evidently.test_preset
✅ evidently.tests

Python version: 3.11.11 | packaged by conda-forge | (main, Dec  5 2024, 08:47:03) [Clang 18.1.8 ]


In [1]:
import pandas as pd
from evidently.report import Report
from evidently.metric_preset import DataQualityPreset, DataDriftPreset
from evidently.test_suite import TestSuite
from evidently.test_preset import DataDriftTestPreset
from evidently.tests import TestNumberOfColumns, TestNumberOfRows, TestNumberOfMissingValues, TestShareOfMissingValues
from evidently import metrics


# ===== 1. CHARGEMENT ET PRÉPARATION DES DONNÉES =====

# Chargement du fichier CSV
paris_df = pd.read_csv("weather_paris.csv")

# Conversion du timestamp en datetime
paris_df['datetime'] = pd.to_datetime(paris_df['datetime'])

# Tri par date
paris_df = paris_df.sort_values('datetime')

print(f"Données chargées: {len(paris_df)} lignes")
print(f"Période: {paris_df['datetime'].min()} à {paris_df['datetime'].max()}")
print(f"\nColonnes disponibles:\n{paris_df.columns.tolist()}")


# ===== 2. DIVISION DES DONNÉES EN RÉFÉRENCE ET ACTUEL =====

# Diviser les données en deux périodes pour la détection de drift
# Référence: première moitié (données "anciennes")
# Current: deuxième moitié (données "actuelles")

split_idx = len(paris_df) // 2

reference_data = paris_df.iloc[:split_idx].copy()
current_data = paris_df.iloc[split_idx:].copy()

print(f"\nDonnées de référence: {len(reference_data)} lignes ({reference_data['datetime'].min()} à {reference_data['datetime'].max()})")
print(f"Données actuelles: {len(current_data)} lignes ({current_data['datetime'].min()} à {current_data['datetime'].max()})")


# ===== 3. SÉLECTION DES COLONNES NUMÉRIQUES POUR L'ANALYSE =====

# Colonnes numériques pertinentes pour l'analyse de drift
numerical_columns = [
    'temp', 'feels_like', 'pressure', 'humidity', 
    'dew_point', 'clouds', 'visibility', 'wind_speed', 
    'wind_deg', 'rain_1h'
]

# Vérification que les colonnes existent
numerical_columns = [col for col in numerical_columns if col in paris_df.columns]

print(f"\nColonnes numériques analysées: {numerical_columns}")


# ===== 4. CRÉATION DU RAPPORT DE DRIFT =====

print("\n" + "="*60)
print("GÉNÉRATION DU RAPPORT DE DRIFT")
print("="*60)

# Rapport complet avec métriques de drift
data_drift_report = Report(metrics=[
    DataDriftPreset(columns=numerical_columns),
    DataQualityPreset(),
])

data_drift_report.run(
    reference_data=reference_data,
    current_data=current_data
)

# Sauvegarde du rapport HTML
report_path = "weather_paris_drift_report.html"
data_drift_report.save_html(report_path)
print(f"\n✓ Rapport sauvegardé: {report_path}")


# ===== 5. CRÉATION DE LA TEST SUITE =====

print("\n" + "="*60)
print("GÉNÉRATION DE LA TEST SUITE")
print("="*60)

# Test suite pour valider les drifts
data_drift_test_suite = TestSuite(tests=[
    DataDriftTestPreset(columns=numerical_columns),
    TestNumberOfColumns(),
    TestNumberOfRows(),
    TestNumberOfMissingValues(),
    TestShareOfMissingValues(),
])

data_drift_test_suite.run(
    reference_data=reference_data,
    current_data=current_data
)

# Sauvegarde de la test suite
test_suite_path = "weather_paris_test_suite.html"
data_drift_test_suite.save_html(test_suite_path)
print(f"\n✓ Test Suite sauvegardée: {test_suite_path}")


# ===== 6. ANALYSE DES RÉSULTATS =====

print("\n" + "="*60)
print("RÉSUMÉ DES RÉSULTATS")
print("="*60)

# Extraction des résultats de la test suite
test_results = data_drift_test_suite.as_dict()

# Comptage des tests réussis/échoués
tests_summary = {
    'passed': 0,
    'failed': 0,
    'warning': 0,
    'error': 0
}

if 'tests' in test_results:
    for test in test_results['tests']:
        status = test.get('status', '').lower()
        if status in tests_summary:
            tests_summary[status] += 1

print(f"\nRésultats des tests:")
print(f"  ✓ Réussis: {tests_summary['passed']}")
print(f"  ✗ Échoués: {tests_summary['failed']}")
print(f"  ⚠ Warnings: {tests_summary['warning']}")
print(f"  ⚡ Erreurs: {tests_summary['error']}")


# ===== 7. CRÉATION DE RAPPORTS SPÉCIFIQUES PAR MÉTRIQUE =====

print("\n" + "="*60)
print("ANALYSES SPÉCIFIQUES PAR MÉTRIQUE")
print("="*60)

# Rapport détaillé pour chaque colonne numérique importante
for col in ['temp', 'humidity', 'pressure', 'wind_speed']:
    if col in numerical_columns:
        specific_report = Report(metrics=[
            metrics.ColumnDriftMetric(column_name=col),
            metrics.ColumnSummaryMetric(column_name=col),
            metrics.ColumnDistributionMetric(column_name=col),
        ])
        
        specific_report.run(
            reference_data=reference_data,
            current_data=current_data
        )
        
        specific_path = f"weather_paris_{col}_analysis.html"
        specific_report.save_html(specific_path)
        print(f"  ✓ Rapport {col}: {specific_path}")


# ===== 8. AFFICHAGE DES STATISTIQUES DESCRIPTIVES =====

print("\n" + "="*60)
print("STATISTIQUES DESCRIPTIVES")
print("="*60)

print("\nDonnées de référence:")
print(reference_data[numerical_columns].describe())

print("\nDonnées actuelles:")
print(current_data[numerical_columns].describe())


print("\n" + "="*60)
print("ANALYSE TERMINÉE !")
print("="*60)
print(f"\nFichiers générés:")
print(f"  1. {report_path}")
print(f"  2. {test_suite_path}")
print(f"  3. Rapports spécifiques par métrique")
print("\nOuvrez ces fichiers HTML dans votre navigateur pour visualiser les résultats.")

Données chargées: 1350 lignes
Période: 2021-09-20 10:00:00 à 2025-05-31 22:00:00

Colonnes disponibles:
['timestamp', 'datetime', 'temp', 'feels_like', 'pressure', 'humidity', 'dew_point', 'clouds', 'visibility', 'wind_speed', 'wind_deg', 'weather_main', 'weather_description', 'rain_1h']

Données de référence: 675 lignes (2021-09-20 10:00:00 à 2023-07-26 10:00:00)
Données actuelles: 675 lignes (2023-07-27 10:00:00 à 2025-05-31 22:00:00)

Colonnes numériques analysées: ['temp', 'feels_like', 'pressure', 'humidity', 'dew_point', 'clouds', 'visibility', 'wind_speed', 'wind_deg', 'rain_1h']

GÉNÉRATION DU RAPPORT DE DRIFT

✓ Rapport sauvegardé: weather_paris_drift_report.html

GÉNÉRATION DE LA TEST SUITE

✓ Test Suite sauvegardée: weather_paris_test_suite.html

RÉSUMÉ DES RÉSULTATS

Résultats des tests:
  ✓ Réussis: 0
  ✗ Échoués: 0
  ⚠ Warnings: 0
  ⚡ Erreurs: 0

ANALYSES SPÉCIFIQUES PAR MÉTRIQUE
  ✓ Rapport temp: weather_paris_temp_analysis.html
  ✓ Rapport humidity: weather_paris_humidi